# ETF CLASSIFICATION 관련 코드 

In [2]:
import pandas as pd
import pymysql
import pandas_datareader.data as data
from datetime import datetime 
import time
import FinanceDataReader as fdr
import numpy as np
from dateutil.relativedelta  import relativedelta
import scipy.optimize as optm
import json 
import urllib.request
import seaborn as sns
from matplotlib import pyplot as plt
import warnings

C:\Anaconda\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


### sql 접속

In [34]:
# 접속
stock_db = pymysql.connect(
    user='crawler',
    passwd='crawler123!',
    host='137.68.198.34', 
    port=3306,
    db='stockdata', 
    charset='utf8'
    )

### 시총 500억이상 & 같은 분야별 시총이 가장 큰 ETF 가져오기

In [157]:
sql = "select ticker, ticker_name, net_asset from tb_etf_info"

cur = stock_db.cursor()
cur.execute(sql)
stock_db.commit()

result = cur.fetchall()

In [158]:
etf_df = pd.DataFrame(result)
etf_df.columns = ['ticker', 'ticker_name', 'net_asset']

# 시가총액 500억 이상인 ETF 필터
etf_df_2 = etf_df[ etf_df['net_asset'] > 500 ]
etf_df_3 = etf_df_2.set_index('ticker_name')
select_etf = pd.concat([etf_df_2['ticker_name'], etf_df_2['net_asset']], axis = 1)

# 같은 분야별 ETF 시가총액이 가장 큰 ETF 필터
select_etf_2 =  select_etf.groupby([select_etf['ticker_name'].map(lambda x: x.split(' ')[1]),
                     select_etf['ticker_name']]).max().sort_values(by = 'net_asset', ascending = False)
select_etf_3 = select_etf_2.unstack()
select_etf_4 = pd.DataFrame(select_etf_3.idxmax(axis=1).values).applymap(lambda x : x[1])
select_etf_4.columns = ['ticker_name']
select_etf_5 = select_etf_4.set_index('ticker_name')

select_etf_6 = pd.merge(left = etf_df_3,right = select_etf_5, on = 'ticker_name', how = 'right' )

### 고객 설문데이터 불러오기

In [170]:
sql = """
select user_id, srno, surv_date, period
from tb_surv
group by user_id, srno
order by user_id, srno
"""
cur = stock_db.cursor()
cur.execute(sql)
stock_db.commit()

result = cur.fetchall()

In [171]:
tb_surv = pd.DataFrame(result)

In [172]:
tb_surv.columns = ['user_id', 'srno', 'surv_date', 'period']

In [173]:
#db에서 가져온 고객과 설문 정보
user_id = tb_surv.user_id[15]
srno = tb_surv.srno[15]
surv_date = tb_surv.surv_date[15]
surv_period = tb_surv.period[15]

### DB에서 종가데이터 가져오기 (db에 있는 날짜 전부)

In [174]:
start = '2018-10-01'
end = '2020-08-14'

ticker = '(' + str(select_etf_6['ticker'].tolist())[1:-1] + ')'

In [175]:
sql = "SELECT ticker, k_date, close  FROM TB_ETF_PRICE WHERE TICKER IN " + ticker + ' AND '\
+ "K_DATE BETWEEN" + " DATE('" +  start + "')" + " AND " + "DATE('" + end + "')"
cur = stock_db.cursor()
cur.execute(sql)
stock_db.commit()

result = cur.fetchall()

In [176]:
tb_etf_price = pd.DataFrame(result)
tb_etf_price.columns = ['ticker', 'k_date', 'close']

tb_etf_price_2 = tb_etf_price.set_index(['k_date','ticker']).unstack()
tb_etf_price_3 = tb_etf_price_2.xs('close',axis=1)
tb_etf_price_3.index.name = 'Date'

# 2018-10-01 이후에 상장된 ETF 제외
etf_close = tb_etf_price_3.dropna(axis=1)
etf_close_2 = etf_close.applymap(lambda x : int(x))

In [177]:
start

'2018-10-01'

In [178]:
etf_close_2

ticker,069500,091170,102780,102970,108450,114260,114800,122260,122630,130680,...,278240,278530,278540,282000,285000,292150,292190,302190,305540,305720
Date,,,,,,,,,,,,,,,,,,,,,
2018-10-01,28816,7688,7008,6421,10608,55038,6410,98589,14845,5630,...,16910,9220,9140,99905,9379,8560,13504,49843,9906,9963
2018-10-02,28467,7647,6925,6221,10472,55063,6490,98594,14480,5790,...,15675,9100,9000,99865,9211,8415,13321,49883,9742,9807
2018-10-04,28027,7764,6842,6056,10341,55004,6595,98604,14015,5820,...,15545,8965,8830,100045,9088,8241,13125,49706,9634,9523
2018-10-05,27984,7755,6770,6009,10229,54959,6600,98604,13980,5755,...,14900,8945,8820,100105,9049,8241,13086,49686,9312,9331
2018-10-08,27844,7742,6755,5944,10156,54974,6630,98599,13860,5660,...,14515,8915,8785,100110,9039,8241,13010,49681,9273,9228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-10,31700,5380,7720,6610,12435,58160,5415,102200,15420,2050,...,13850,10155,10065,99040,11960,10970,14840,54220,12910,13095
2020-08-11,32100,5580,7825,6535,12630,58160,5345,102195,15840,2060,...,13750,10295,10250,99060,11880,11115,15015,54270,12885,13290
2020-08-12,32275,5695,8000,6500,12910,58150,5320,102195,15995,2055,...,13110,10340,10255,99060,11990,11030,15010,54160,12875,13270


In [179]:
# 영업일 아닌경우 전날로 변경
surv_date = datetime.strptime(datetime.strftime(surv_date, '%Y-%m-%d'), '%Y-%m-%d') 
for i in list(range(10)) :
    if ( surv_date in etf_close_2.index.tolist() ) == True :
        break
    if ( surv_date in etf_close_2.index.tolist() ) == False :
        surv_date = surv_date - relativedelta( days= 1 )

### USER_CLASS 가져오기

In [177]:
sql = "select class from tb_user_class where user_id = "\
+ str(user_id) + " and " + "srno = " + str(srno)

cur = stock_db.cursor()
cur.execute(sql)
stock_db.commit()

result = cur.fetchall()

In [178]:
user_class = result[0][0]

### 통계 집계 테이블 만들기

In [179]:
etf_ret

ticker,069500,091170,102780,102970,108450,114260,114800,122260,122630,130680,...,273140,275980,278240,278530,278540,282000,285000,292150,292190,302190
Date,,,,,,,,,,,,,,,,,,,,,
2018-09-04,0.004387,0.000000,0.003646,0.000650,0.004324,0.000544,-0.005397,0.000142,0.009962,0.008467,...,0.000151,-0.004246,0.035244,0.006037,0.004422,-0.000101,0.011129,0.007861,0.003224,0.000180
2018-09-05,-0.012928,-0.017764,-0.008428,-0.017854,-0.010959,-0.000181,0.014729,-0.000091,-0.026531,-0.013993,...,0.000201,-0.000426,-0.035820,-0.013639,-0.014860,-0.000302,-0.016001,-0.017550,-0.011137,-0.000380
2018-09-06,-0.001451,-0.006956,-0.003664,-0.012395,-0.003759,0.000798,0.002292,0.000193,-0.003843,-0.009461,...,0.000000,-0.007679,-0.011360,-0.003319,-0.003911,-0.000453,-0.012325,-0.002218,-0.001436,0.002459
2018-09-07,-0.005636,0.008966,-0.007060,0.010877,-0.007746,-0.000344,0.003049,-0.000142,-0.010523,-0.006686,...,0.000100,-0.006879,0.002174,-0.002775,-0.009534,0.000654,-0.014367,-0.010765,-0.003633,-0.000399
2018-09-10,0.002210,0.003055,0.006369,0.007780,0.009708,0.000072,0.000760,-0.000051,0.003545,0.004808,...,-0.000201,-0.000866,-0.000620,0.000556,0.003398,-0.000050,0.005745,-0.003430,-0.001823,0.000399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-10,0.014887,0.029665,0.004554,0.072182,0.000805,-0.000516,-0.014559,0.000049,0.028000,-0.004854,...,-0.000049,0.002786,0.007639,0.013473,0.011050,0.000101,0.008007,0.004579,0.015743,-0.001749
2020-08-11,0.012618,0.037175,0.013601,-0.011346,0.015682,0.000000,-0.012927,-0.000049,0.027237,0.004878,...,0.000000,-0.007292,-0.007220,0.013786,0.018381,0.000202,-0.006689,0.013218,0.011792,0.000922
2020-08-12,0.005452,0.020609,0.022364,-0.005356,0.022169,-0.000172,-0.004677,0.000000,0.009785,-0.002427,...,0.000000,-0.013291,-0.046545,0.004371,0.000488,0.000000,0.009259,-0.007647,-0.000333,-0.002027


In [180]:
etf_ret = etf_close_2.pct_change().dropna()

In [181]:
befor_1y = etf_ret.rolling(252).std().dropna().index[0]

In [180]:
# start_date, end_date 설정
surv_period_int = int(surv_period[0])

if surv_period_int == 1:
    start_date = datetime.strftime(surv_date-relativedelta(years= surv_period_int ),'%Y-%m-%d')
else :
    start_date = datetime.strftime(surv_date-relativedelta(months= surv_period_int ),'%Y-%m-%d')
end_date =  datetime.strftime(surv_date, '%Y-%m-%d')

### 채권금리 데이터 만들기

In [122]:
#채권금리 크롤링해오기
KR1YT = 'KR1YT=RR'
risk_free_df = fdr.DataReader(KR1YT, '2018-01-02', '2020-08-14')

In [123]:
risk_free_df_2 = risk_free_df.reset_index()
risk_free_df_2['Ticker'] = ['KR1YT'] * len(risk_free_df)
risk_free_df_2['Volume'] = 0 
risk_free_df_3 = risk_free_df_2.reindex(columns = ['Ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Change'])

In [124]:
risk_free_df_3['Date'] = risk_free_df_3['Date'].astype(str)

In [128]:
rf_insert = np.vstack(np.array(risk_free_df_3)).tolist()

### 채권금리 DB에서 가져오기

In [204]:
sql = """
select avg(close)
from tb_stk_price
where ticker = 'KR1YT'
And k_date between '%s' and '%s'
""" % (start_date, end_date)

cur = stock_db.cursor()
cur.execute(sql)
stock_db.commit()

cur.execute(sql)
result = cur.fetchall()
risk_free = float(result[0][0])

# risk free rate 구하기

if surv_period == '1Y' :
    risk_free = risk_free * 0.01
elif surv_period == '3M' :
    risk_free = (risk_free * 0.01) / 4
elif surv_period == '6M' :
    risk_free = (risk_free * 0.01) / 2

In [186]:
# 기간별 수익률의 표준편차 계산 
std_3M = etf_ret.rolling(63).std().loc[befor_1y:].stack()
std_6M = etf_ret.rolling(126).std().loc[befor_1y:].stack()
std_1Y = etf_ret.rolling(252).std().dropna().stack()

# 기간별 수익률의 평균 계산 
mean_3M = etf_ret.rolling(63).mean().loc[befor_1y:].stack()
mean_6M = etf_ret.rolling(126).mean().loc[befor_1y:].stack()
mean_1Y = etf_ret.rolling(252).mean().dropna().stack()

#기간별 Sharp Ratio 계산
rf_list = [risk_free] * len(std_3M)
rf_Series_ = std_3M.copy()
rf_Series_.iloc[:] = rf_list

SR_3M = ( ( mean_3M * 63 ) - rf_Series_) / ( std_3M * np.sqrt(63) )
SR_6M = ( ( mean_6M * 126 ) - rf_Series_) / ( std_6M * np.sqrt(126) )
SR_1Y = ( ( mean_1Y * 252 )  - rf_Series_) / ( std_1Y * np.sqrt(252) )

In [189]:
# 1. Series의 형태로 되있는 멀티인덱스를 해제 / 2. 날짜에 기간 추가해주는 함수
def reset_index_add_period(df_period,period) :
    df_period = pd.DataFrame(df_period.copy()).reset_index()
    df_period['Date'] = df_period['Date'].map(lambda x : x.strftime('%Y-%m-%d') + '_' + period)
    return df_period

In [190]:
# 멀티인덱스를 해제 & 날짜에 기간 추가
std_3M = reset_index_add_period(std_3M, '3M')
std_6M = reset_index_add_period(std_6M, '6M')
std_1Y = reset_index_add_period(std_1Y, '1Y')

mean_3M = reset_index_add_period(mean_3M, '3M')
mean_6M = reset_index_add_period(mean_6M, '6M')
mean_1Y = reset_index_add_period(mean_1Y, '1Y')

SR_3M = reset_index_add_period(SR_3M, '3M')
SR_6M = reset_index_add_period(SR_6M, '6M')
SR_1Y = reset_index_add_period(SR_1Y, '1Y')

# class 구분을 위한 std 복사
class_3M = std_3M.copy()
class_6M = std_6M.copy()
class_1Y = std_1Y.copy()

# classification을 위한 key값 달아주기
class_3M['key'] = list(range(len(class_3M)))
class_6M['key'] = list(range(len(class_6M)))
class_1Y['key'] = list(range(len(class_1Y)))

In [191]:
#기간별 수익률의 표준편차에 따른 classification해주는 함수

def period_class(df_period) :
    class_df = df_period.set_index('Date')

    qt_20 = pd.DataFrame(df_period.groupby('Date')[0].quantile(0.2))
    qt_40 = pd.DataFrame(df_period.groupby('Date')[0].quantile(0.4))
    qt_60 = pd.DataFrame(df_period.groupby('Date')[0].quantile(0.6))
    qt_80 = pd.DataFrame(df_period.groupby('Date')[0].quantile(0.8))
    qt_100 = pd.DataFrame(df_period.groupby('Date')[0].quantile(1))

    qt_20.columns = ['20%']
    qt_40.columns = ['40%']
    qt_60.columns = ['60%']
    qt_80.columns = ['80%']
    qt_100.columns = ['100%']

    qt_compar = pd.merge(left = class_df,right = qt_20, on = 'Date', how = 'right' )
    qt_compar2 = pd.merge(left = qt_compar,right = qt_40, on = 'Date', how = 'right' )
    qt_compar3 = pd.merge(left = qt_compar2,right = qt_60, on = 'Date', how = 'right' )
    qt_compar4 = pd.merge(left = qt_compar3,right = qt_80, on = 'Date', how = 'right' )
    qt_compar5 = pd.merge(left = qt_compar4,right = qt_100, on = 'Date', how = 'right' )

    qt_compar5['class_20'] = qt_compar5[0] <= qt_compar5['20%']
    qt_compar5['class_20'] = qt_compar5['class_20'].map(lambda x : 1 if x == True else 0)
    
    # quantile 추가해주면서 해당 class에 1을 넣어주는 함수 (아닌 class는 0)
    def classify(add_class) : 
        qt_compar5[add_class] = qt_compar5[0] <= qt_compar5[add_class.split('_')[1] + '%']
        qt_compar5[add_class] = qt_compar5[add_class].map(lambda x : 0 if x == True else 1)
        if add_class == 'class_40' : 
            qt_compar5[add_class] = qt_compar5['class_20'] + qt_compar5['class_40'] == 0
        if add_class == 'class_60' :
            qt_compar5[add_class] = qt_compar5['class_20'] + qt_compar5['class_40'] + qt_compar5['class_60'] == 0
        if add_class == 'class_80' :
            qt_compar5[add_class] = qt_compar5['class_20'] + qt_compar5['class_40'] + qt_compar5['class_60'] + qt_compar5['class_80'] == 0
        if add_class == 'class_100' :
            qt_compar5[add_class] = qt_compar5['class_20'] + qt_compar5['class_40'] + qt_compar5['class_60'] + qt_compar5['class_80'] + qt_compar5['class_100']== 0
        qt_compar5[add_class] = qt_compar5[add_class].map(lambda x : 1 if x == True else 0)

    classify('class_40')
    classify('class_60')
    classify('class_80')
    classify('class_100')
    
    # 1로 분류되어있는 각 class별로 1, 2, 3, 4, 5 label을 달아주는 함수
    def label(class_name, num) : 
        qt_compar5[class_name] = qt_compar5[class_name] == 1
        qt_compar5[class_name] = qt_compar5[class_name].map(lambda x : num if x == True else 0)

    label('class_40', 2)
    label('class_60', 3)
    label('class_80', 4)
    label('class_100', 5)

    # 열로 구분되어있는 class를 하나로 합쳐주기
    class_df_2 = pd.concat([qt_compar5.loc[:, 'key'], qt_compar5.loc[:, 'class_20':'class_100']], axis = 1)
    class_df_2['class'] = qt_compar5['class_20'] + qt_compar5['class_40'] + qt_compar5['class_60'] +qt_compar5['class_80'] + qt_compar5['class_100']
    class_df_3 = pd.concat([class_df_2.loc[:, 'key'], class_df_2.loc[:, 'class']], axis=1)
    class_df_4 = pd.merge(left = df_period,right = class_df_3, on = 'key', how = 'right' )

    class_df_5 = class_df_4.set_index(['Date','ticker'])['class']
    return class_df_5

In [192]:
# 3개월, 6개월 , 1년 df 밑으로 합치기
total_class = pd.DataFrame(pd.concat([period_class(class_3M),
                                     period_class(class_6M),
                                     period_class(class_1Y)])).reset_index().set_index(['Date','ticker'])
total_std = pd.concat([std_3M, std_6M, std_1Y]).set_index(['Date','ticker'])
total_mean = pd.concat([mean_3M, mean_6M, mean_1Y]).set_index(['Date','ticker'])
total_SR = pd.concat([SR_3M, SR_6M, SR_1Y]).set_index(['Date','ticker'])

In [193]:
total_4 = pd.concat([ total_class, total_std, total_mean, total_SR ], axis=1)
total_4 = total_4.reset_index()
total_4.columns = ['date', 'ticker', 'class','std','mean','SR']

total_5 = total_4.copy()
total_5['date'] =  total_4['date'].map(lambda x : x[:10])
total_5['period'] = total_4['date'].map(lambda x : x[11:14])

In [194]:
total_5 = total_5.reindex(columns = ['date', 'period', 'ticker', 'class',	'std', 'mean', 'SR'])

In [195]:
insert_list = np.vstack(np.array(total_5)).tolist()